Generate the matrix data for data collection:
- Define matrix types (dense, sparse, etc.).
- Collect performance metrics like execution time and memory usage.

In [4]:
import numpy as np
import pandas as pd
import time

# Generate dense matrix
dense_matrix_100x100 = np.random.rand(100, 100)

# Generate sparse matrix (10% non-zero elements)
sparse_matrix_100x100 = np.random.rand(100, 100)
sparse_matrix_100x100[sparse_matrix_100x100 < 0.9] = 0  # Setting 90% to zero

# Function to measure time taken for multiplication
def measure_time(matrix1, matrix2):
    start_time = time.time()
    np.matmul(matrix1, matrix2)
    end_time = time.time()
    return end_time - start_time

def main():
    # df = pd.DataFrame(dense_matrix_100x100)
    # print(df)
    execution_time = measure_time(dense_matrix_100x100, dense_matrix_100x100)
    print(f"Execution Time: {execution_time:.4f} seconds")

if __name__ == "__main__":
    main()


Execution Time: 0.0002 seconds
